## TASK 1 
Write a function compute the CF  fj(x,v,T,ϕ)  in Heston model as in Heston (1993). Recall that the imaginary number  i  is coded as  1j  in Python. Check your function with the following inputs. You should get as ouput: (0.9999999999892609+4.632064148415667e-06j)

In [1]:
import numpy as np
import math 

# Numerical example to check your coded CF 
# Option features
S0 = 100; K = 100
T = 0.5       # Maturity
# we do not have any t here so i guess we pick tau = 0.5 - 0 = 0.5
r = 0.03;   # risk-free rate; 
# Heston parameters
tau = .5
kappa = 5; sigma = 0.5;  rho   = -0.8;  theta = 0.05; v0 = 0.05; lam = 0; q=0 #dividend yield
# The phi variable value
phi= 0.000001

def heston_CF(phi, kappa, theta, lam, rho, sigma, T, S0, r, v0, Pj):
    if Pj == 1: #make an if statement for the different Pj and give the formulas
        u  = 0.5
        b = kappa + lam - rho*sigma
    else:
        u = -0.5
        b = kappa + lam
    # below is all the coding we need. Basically there is nothing more than
    # trying to implement the formulas. I came against some problems so 
    # I had to split a bit the formulas to make them work. It looks like it is 
    # a bit sensitive with the parantheses or maybe I could not make it work in a big
    # equation
    dNoSquareRoot = (rho*sigma*1j*phi-b)**2-(sigma**2)*(2*u*1j*phi-phi**2)
    d = np.sqrt(dNoSquareRoot)
    g = (b-rho*sigma*1j*phi+d)/(b-rho*sigma*1j*phi-d)
    D_PARTA = (b-rho*sigma*1j*phi+d)/(sigma**2)
    D_PARTB = (1-np.exp(d*tau))/(1-g*np.exp(d*tau))
    D = D_PARTA*D_PARTB
    a = kappa*theta
    x = np.log(S0)
    C_PARTA = (r-q)*1j*phi*tau 
    C_PARTB = (a/sigma**2)*((b-rho*sigma*1j*phi+ d)*tau-2*np.log((1-g*np.exp(d*tau))/(1-g)))
    C = C_PARTA + C_PARTB
    cf = np.exp(C + D*v0 + 1j*phi*x)         
    return cf                         

In [2]:
heston_CF(phi, kappa, theta, lam, rho, sigma, T, S0, r, v0, 1) # the correct output according to the
# task 1 headline
# (0.9999999999892609+4.632064148415667e-06j)

(0.9999999999892609+4.632064148415667e-06j)

## TASK 2 
Write a function to give the integrand in Formula (18) (for computing  Pj s) in Heston (1993). Note that if  f  is a complex number in Python,  f.real  gives the real part of it. Check your fucnction using the above example, you should get as output  0.026893962444189606


In [3]:
def heston_Integrand(phi, kappa, theta, lam, rho, sigma, T, S0, K, r, v0, Pj): # define the function
    fj = heston_CF(phi, kappa, theta, lam, rho, sigma, T, S0, r, v0, Pj) # we already have the f inside the
    # formula
    remaining_part = (np.exp(-1j*phi*np.log(K))*fj/(1j*phi)).real #define the remaining part of the formula
    return remaining_part # return it

In [4]:
heston_Integrand(phi, kappa, theta, lam, rho, sigma, T, S0, K, r, v0, 1) # call the function and we see
# that we actually get the desired output
# 0.026893962444189606

0.026893962444189606

## TASK 3 
Write a function to compute  P1  and  P2  i.e. Equation (18) in Heston (1993).
I1 = 0.40968542618813575 (The definte integral for P1)
I2 = 0.22059617207452004 (The definte integral for P2)
P1= 0.6304069213811032
P2= 0.5702179424256204

In [5]:

import scipy.integrate as sci # import the scipy.integrate
def P_1_2(kappa, theta, lam, rho, sigma, T, S0, K, r, v0): 
    args1 = ( kappa, theta, lam, rho, sigma, T, S0, K, r, v0,1) #set the arguments for the sci.quad
    args2 = ( kappa, theta, lam, rho, sigma, T, S0, K, r, v0,2) #set the arguments for the sci.quad
    I1 = sci.quad(heston_Integrand,0.000001,50, args=args1)[0] #get the first element of the tuple
    I2 = sci.quad(heston_Integrand,0.000001,50, args=args2)[0] #get the first element of the tuple
    P1 = 1/2 + (1/math.pi)*I1 # calculate p1 and p2
    P2 = 1/2 + (1/math.pi)*I2
    return I1,I2,P1,P2
print(P_1_2(kappa, theta, lam, rho, sigma, T, S0, K, r, v0)) # here
# and we get the desired outputs
# I1 = 0.40968542618813575 (The definte integral for P1)
# I2 = 0.22059617207452004 (The definte integral for P2)
# P1= 0.6304069213811032
# P2= 0.5702179424256204

(0.40968542618813575, 0.22059617207452004, 0.6304069213811032, 0.5702179424256204)


## TASK 4
Finally write a function to compute the call price using Equation (10) in Heston (1993). You should get the call price approximately 6.8678.

In [6]:
def hestonPrice(phi, kappa, theta, lam, rho, sigma, tau, S0, K, r, v0):  #define here the function to calculate the price according to the heston
    (P1,P2) = P_1_2(kappa, theta, lam, rho, sigma, T, S0, K, r, v0)[2:4]
    c = S0*P1 - K*np.exp(-r*T)*P2
    return c
call = hestonPrice(phi, kappa, theta, lam, rho, sigma, tau, S0, K, r, v0)
print(call) #correct pricing
# approximately 6.8678

6.867841812173282


## TASK 5
Now compute the Heston price for another numerical example. What price you get? Does this price make sense?

# Example two
S = 7; K = 10
tau = 3/12        # Maturity
r = 0.0;   # risk-free rate; Dividend yield
# Heston parameters
kappa = 5; sigma = 0.3;  rho   = -0.9;  theta = 0.07; v0 = 0.07; lam = 0;  T=tau

In [7]:
# Example two
S1= 7; K1 = 10
tau1 = 3/12        # Maturity
r1 = 0.0;   # risk-free rate; Dividend yield
# Heston parameters
kappa1 = 5; sigma1 = 0.3;  rho1   = -0.9;  theta1 = 0.07; v01 = 0.07; lam1 = 0; 
print(phi, kappa1, theta1, lam1, rho1, sigma1, tau1, S1, K1, r1, v01)
c_Example2 = hestonPrice(phi, kappa1, theta1, lam1, rho1, sigma1, tau1, S1, K1, r1, v01)
print(c_Example2)
# Yes it should be making sense since the strike and spot price are so low. We should expect a really 
# low value. At least that's what I think. There is the possibility that my variables are a bit mixed up
# I did a lot of tests and I had a quite problem with this task since I had to change my code a lot
# to make a function that accepts arguments and it is dynamic. Task 4 works fine but here maybe 
# the function is getting confused with some older variable and the correct result could be different than
# the one I get. I hope I make sense. I changed the variables' name by adding a "1"
# because there were conflicts with the other variables of Task 4.

1e-06 5 0.07 0 -0.9 0.3 0.25 7 10 0.0 0.07
0.0033476642172580173


## TASK 6
Read article The little Heston trap by Albrecher et al. (2007). Write a summary on the issues pointed out in this article. You don't need to understand the very technical part on complex integrals, just explain the best you can what is "the little Heston Trap", and what is solution to avoiding this trap.

#### Summary
There are two different equations for the Heston characteristic function. These two equations are in fact equavelant as it is observed in the correspoding paper. However, the biggest issue is "d"'s sign where is flipped around in these two equations meaning that it can be either negative or positive. This occurs from the fact that "d" is a complex root that has two different values and the second value is given by substracting the first one. Additionally, the branch cut seems to be the main problem here. A branch cut is where the a function is discontinuous in the complex plane. As someone increases "u" in ϕ_1(u,t) the main value of the complex square root basically moves to the negative real axis. This causes a discontinuous function to appear which is the problem of the numerical calculations. Further, some of the mispricings that are caused by that discontinuous function are not always noticable. 

A solution to tackle these mispricings is to use the ϕ_2(u,t) along the main value of the square root. That way numerical calculations are more stable since the discontinuities do not appear. Another one, more recent, is to restrict certain parameters. 